In [2]:
import os
import numpy as np
import collections
import pickle
import argparse
import tensorflow as tf
import time
import matplotlib.pyplot as plt
%matplotlib inline

/software/Anaconda3-5.0.1-el7-x86_64/envs/DL_GPU_cuda_9.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Set parameters

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', default='./')
parser.add_argument('--save_dir', default='./')
# Dimension of hidden layer variables h and c
parser.add_argument('--num_units', default=128*2)
parser.add_argument('--batch_size', default=64)
# Number of steps in each batch for training
parser.add_argument('--num_steps', default=75)
parser.add_argument('--num_epochs', default=30)
# Time step
parser.add_argument('--lr', default=0.002)
# Number of possible inputs/outputs 
parser.add_argument('--num_chars')
parser.add_argument('--num_batches',default=20)
args, unparsed = parser.parse_known_args()

# Define a timer

In [4]:
def timer(start, end):
    hrs, rem = divmod(end-start, 3600)
    mins, secs = divmod(rem, 60)
    print('{:0>2} hours {:0>2} minutes {:05.2f} seconds'.format(int(hrs), int(mins), secs))

# Load text, vreate vocabulary, define batches from text.

In [5]:
class TextLoader():

    def __init__(self, data_dir, batch_size=64, seq_length=50, encoding='utf-8'):
        self.data_dir = data_dir
        self.encoding = encoding
        self.batch_size = batch_size
        self.seq_length = seq_length

        self.input_file = os.path.join(data_dir, 'tinyshakespeare.txt')
        self.vocab_file = os.path.join(data_dir, 'vocab.pkl')
        # Numeric file of characters translated to indices.
        self.tensor_file = os.path.join(data_dir, 'data.npy')
        
        if not (os.path.exists(self.vocab_file) and os.path.exists(self.tensor_file)):
            print('it seems we havent processed the text data yet: reading the shakespear')
            self.preprocess(self.input_file, self.vocab_file, self.tensor_file)
        else:
            print('there are preprocessed data - lets load it')
            self.load_preprocessed(self.vocab_file, self.tensor_file)

        self.create_batches()
        self.reset_batch_pointer()

    # Create numeric file.
    def preprocess(self, input_file=None, vocab_file=None, tensor_file=None, saveit=True):
        if input_file is not None:
            self.input_file = input_file
        if vocab_file is not None:
            self.vocab_file = vocab_file
        if tensor_file is not None:
            self.tensor_file = tensor_file

        with open(self.input_file, 'r') as f:
            data = f.read()
        #data = data.lower()
        self.total_length = len(data)
        counter = collections.Counter(data)
        count_pairs = sorted(counter.items(), key=lambda x: -x[1])
        self.chars, _ = zip(*count_pairs)
        self.vocab_size = len(self.chars)
        self.vocab_to_idx = dict(zip(self.chars, range(len(self.chars))))
        self.idx_to_vocab = dict(zip(self.vocab_to_idx.values(), self.vocab_to_idx.keys()))

        if saveit:
            with open(self.vocab_file, 'wb') as f:  # saving dictionary so we don't compute it again
                pickle.dump(self.chars, f)
            self.tensor = np.array(list(map(self.vocab_to_idx.get, data)))
            np.save(self.tensor_file, self.tensor)  # saving the numerified data
    # Load numeric file create dictionaries for char2idx and back
    def load_preprocessed(self, vocab_file=None, tensor_file=None):
        if vocab_file is not None:
            self.vocab_file = vocab_file
        if tensor_file is not None:
            self.tensor_file = tensor_file

        with open(self.vocab_file, 'rb') as f:
            self.chars = pickle.load(f)

        # attributes
        self.vocab_size = len(self.chars)
        self.vocab = dict(zip(self.chars, range(self.vocab_size)))
        self.vocab_to_idx = dict(zip(self.chars, range(len(self.chars))))
        self.idx_to_vocab = dict(zip(self.vocab_to_idx.values(), self.vocab_to_idx.keys()))
        self.tensor = np.load(tensor_file)
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))
            
    # tensor size = the length of the entire data sequence
    # divide into batch_size sub sequences and stack
    # cut those by seq_length to produce batches of [batch size, seq_length] sized examples
    def create_batches(self):

        
        self.num_batches = int(self.tensor.size / (self.batch_size * self.seq_length))

        if self.num_batches == 0:
            assert False, 'Not enough data. Make seq_length and/or batch_size smaller'

        self.tensor = self.tensor[:self.num_batches * self.batch_size * self.seq_length]  # so we get an even divide
        xdata = self.tensor
        ydata = np.copy(self.tensor)

        # ydata is one step ahead of x and last item is first item of x 
        # to get sequences of same length    
        ydata[:-1] = xdata[1:] 
        ydata[-1] = xdata[0]

        self.x_batches = np.split(xdata.reshape(self.batch_size, -1), self.num_batches, 1)
        self.y_batches = np.split(ydata.reshape(self.batch_size, -1), self.num_batches, 1)
        
        self.train_num_batches=np.int32(self.num_batches*.8)
        self.test_num_batches=self.num_batches-self.train_num_batches
        self.train_x_batches=self.x_batches[0:self.train_num_batches]
        self.train_y_batches=self.y_batches[0:self.train_num_batches]
        self.test_x_batches=self.x_batches[self.train_num_batches:]
        self.test_y_batches=self.y_batches[self.train_num_batches:]

        # xdata: L length
        # xdata reshaped: batch_size, (L/batch_size) length following natural indexing
        # np.split: into num batches batches along the width(sentence)

    def next_batch_train(self):
        x, y = self.train_x_batches[self.pointer], self.train_y_batches[self.pointer]
        self.pointer += 1
        return x,y
    
    def next_batch_test(self):
        x, y = self.test_x_batches[self.pointer], self.test_y_batches[self.pointer]
        self.pointer += 1
        return x,y

    def reset_batch_pointer(self):
        self.pointer = 0

In [6]:
loader = TextLoader(args.data_dir, batch_size=args.batch_size, seq_length=args.num_steps)
args.num_chars = loader.vocab_size
print('num chars',args.num_chars)
print('num batches',loader.num_batches)

there are preprocessed data - lets load it
num chars 65
num batches 232


# Basic RNN cell. 

In [7]:
class MyBasicRNNCell(tf.contrib.rnn.BasicRNNCell):

    def build(self, inputs_shape):

        input_depth = inputs_shape[1].value
        
        self._kernel = self.add_variable(name="kernel_hidden", shape=[input_depth + self._num_units, self._num_units])
        self._bias = self.add_variable(name="bias_hidden", shape=[self._num_units], initializer=tf.zeros_initializer())
        
        self.built = True

    def call(self, inputs, state):
        """Most basic RNN: output = new_state = act(W * input + U * state + B)."""
        
        output = tf.tanh(tf.matmul(tf.concat([inputs, state], 1), self._kernel) + self._bias)

        return output, output

# RNN cell with one intermediate layer before output hidden layer

In [8]:
class MyBasicRNNCell_d2(tf.contrib.rnn.BasicRNNCell):

    def build(self, inputs_shape):

        input_depth = inputs_shape[1].value
        
        self._kernel = self.add_variable(name="kernel_hidden", shape=[input_depth + self._num_units, self._num_units])
        self._bias = self.add_variable(name="bias_hidden", shape=[self._num_units], initializer=tf.zeros_initializer())
        self._kernel1 = self.add_variable(name="kernel_hidden1", shape=[self._num_units, self._num_units])
        self._bias1 = self.add_variable(name="bias_hidden1", shape=[self._num_units], initializer=tf.zeros_initializer())
        self.built = True

    def call(self, inputs, state):
        """Most basic RNN: output = new_state = act(W * input + U * state + B)."""
        
        inter1 = tf.tanh(tf.matmul(tf.concat([inputs, state], 1), self._kernel) + self._bias)
        output = tf.tanh(tf.matmul(inter1, self._kernel1) + self._bias1)
        return output, output

# RNN cell with 3 intermediate layers before output hidden layer

In [9]:
class MyBasicRNNCell_d4(tf.contrib.rnn.BasicRNNCell):

    def build(self, inputs_shape):

        input_depth = inputs_shape[1].value
        
        self._kernel = self.add_variable(name="kernel_hidden", shape=[input_depth + self._num_units, self._num_units])
        self._bias = self.add_variable(name="bias_hidden", shape=[self._num_units], initializer=tf.zeros_initializer())
        self._kernel1 = self.add_variable(name="kernel_hidden1", shape=[self._num_units, self._num_units])
        self._bias1 = self.add_variable(name="bias_hidden1", shape=[self._num_units], initializer=tf.zeros_initializer())
        self._kernel2 = self.add_variable(name="kernel_hidden2", shape=[self._num_units, self._num_units])
        self._bias2 = self.add_variable(name="bias_hidden2", shape=[self._num_units], initializer=tf.zeros_initializer())
        self._kernel3 = self.add_variable(name="kernel_hidden3", shape=[self._num_units, self._num_units])
        self._bias3 = self.add_variable(name="bias_hidden3", shape=[self._num_units], initializer=tf.zeros_initializer())
        self.built = True

    def call(self, inputs, state):
        """Most basic RNN: output = new_state = act(W * input + U * state + B)."""
        
        inter1 = tf.tanh(tf.matmul(tf.concat([inputs, state], 1), self._kernel) + self._bias)
        inter2 = tf.tanh(tf.matmul(inter1, self._kernel1) + self._bias1)
        inter3 = tf.tanh(tf.matmul(inter2, self._kernel2) + self._bias2)
        output = tf.tanh(tf.matmul(inter3, self._kernel3) + self._bias3)


        return output, output

# Basic LSTM cell

In [10]:
LSTMStateTuple = collections.namedtuple("LSTMStateTuple", ("c", "h"))

class MyBasicLSTMCell(tf.contrib.rnn.BasicLSTMCell):

    def build(self, inputs_shape):

        input_depth = inputs_shape[1].value
        self._kernel = self.add_variable(name="kernel", shape=[input_depth + self._num_units, 4 * self._num_units])
        self._bias = self.add_variable(name="bias", shape=[4 * self._num_units], initializer=tf.zeros_initializer())

        self.built = True

    def call(self, inputs, state):

        one = tf.constant(1, dtype=tf.int32)
        c, h = state

        gate_inputs = tf.matmul(tf.concat([inputs, h], axis=1), self._kernel) + self._bias

        input_gate_weights, input_weights, forget_gate_weights, output_gate_weights = tf.split(
            value=gate_inputs, num_or_size_splits=4, axis=one)

        # forget gating
        forget_bias_tensor = tf.constant(1.0, dtype=forget_gate_weights.dtype)
        forget_gate = tf.sigmoid(forget_gate_weights + forget_bias_tensor)
        gated_memory = c * forget_gate

        # input gating
        processed_new_input = tf.tanh(input_weights)
        input_gate = tf.sigmoid(input_gate_weights)
        gated_input = input_gate * processed_new_input

        # updating memory
        new_c = gated_memory + gated_input

        # output gating
        processed_memory = tf.tanh(new_c)
        output_gate = tf.sigmoid(output_gate_weights)
        new_h = processed_memory * output_gate

        new_state = tf.nn.rnn_cell.LSTMStateTuple(new_c, new_h)

        return new_h, new_state

# Basic LSTM cell without gating of input

In [15]:
LSTMStateTuple = collections.namedtuple("LSTMStateTuple", ("c", "h"))

class MyBasicLSTMCell_lessgate(tf.contrib.rnn.BasicLSTMCell):

    def build(self, inputs_shape):

        input_depth = inputs_shape[1].value
        self._kernel = self.add_variable(name="kernel", shape=[input_depth + self._num_units, 4 * self._num_units])
        self._bias = self.add_variable(name="bias", shape=[4 * self._num_units], initializer=tf.zeros_initializer())

        self.built = True

    def call(self, inputs, state):

        one = tf.constant(1, dtype=tf.int32)
        c, h = state

        gate_inputs = tf.matmul(tf.concat([inputs, h], axis=1), self._kernel) + self._bias

        input_gate_weights, input_weights, forget_gate_weights, output_gate_weights = tf.split(
            value=gate_inputs, num_or_size_splits=4, axis=one)

        # forget gating
        forget_bias_tensor = tf.constant(1.0, dtype=forget_gate_weights.dtype)
        forget_gate = tf.sigmoid(forget_gate_weights + forget_bias_tensor)
        gated_memory = c * forget_gate

        # input gating
        processed_new_input = tf.tanh(input_weights)
        #input_gate = tf.sigmoid(input_gate_weights)
        #gated_input = input_gate * processed_new_input

        # updating memory
        new_c = gated_memory + processed_new_input #gated_input

        # output gating
        processed_memory = tf.tanh(new_c)
        output_gate = tf.sigmoid(output_gate_weights)
        new_h = processed_memory * output_gate

        new_state = tf.nn.rnn_cell.LSTMStateTuple(new_c, new_h)

        return new_h, new_state

# Create network, by unrolling cell to length inputs_list - length of inputs, and the cross entropy loss for prediction

In [10]:
def network(myLSTMCell,inputs,targets):

    with tf.variable_scope('embedding_matrix'):
        embedding = tf.get_variable('embedding', [args.num_chars, args.num_units])
        embedded_inputs = tf.nn.embedding_lookup(embedding, inputs)
        inputs_list = tf.unstack(embedded_inputs, axis=1)  # shape: a list of [batch_size, num_units] length num_steps

    with tf.variable_scope('LSTMCell') as myscope:
        cell = myLSTMCell(args.num_units)
        init_state = cell.zero_state(args.batch_size, tf.float32)
        state = init_state
        outputs = []

        for time_, input in enumerate(inputs_list):
            if time_ > 0:
                myscope.reuse_variables()
           
            output, state = cell(input, state)
            outputs.append(output)
    # All hidden outputs for each batch and every step in the batch are reshaped
    # as one long matrix to be transformed to logits and compared to targets.
        output_reshaped = tf.reshape(tf.concat(outputs, 1), [-1, args.num_units])

        final_state = state

    with tf.variable_scope('regression'):
        W = tf.get_variable('W', [args.num_units, args.num_chars])
        b = tf.get_variable('b', [args.num_chars], initializer=tf.constant_initializer(0.0))
        logits = tf.matmul(output_reshaped, W) + b
        prob = tf.nn.softmax(logits)

    with tf.variable_scope('cost'):
        targets_straightened = tf.reshape(targets, [-1])
        crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, 
                                                    labels=targets_straightened)
        loss = tf.reduce_mean(crossentropy)
        cost = loss/args.batch_size/args.num_steps

    with tf.variable_scope('optimizer'):
        train_step = tf.train.AdamOptimizer(args.lr).minimize(loss)

    with tf.variable_scope('saver'):
        saver = tf.train.Saver()
    return init_state, train_step, loss, final_state, saver, prob

# Run the training

In [11]:
def trainer(myCell,num_batches=None):
    
    start_time = time.time()
    tf.reset_default_graph()
    # Define the placeholders
    with tf.variable_scope('placeholders'):
            inputs = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
            targets = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
    # Create the network
    init_state, train_step, loss, final_state, saver, prob=network(myCell,inputs,targets)
    print('train_num_batches',loader.train_num_batches)
    
    if (num_batches is None):
        num_batches=loader.train_num_batches

    with tf.Session() as sess:
         
        sess.run(tf.global_variables_initializer())
        # computation graph for training
        training_losses = []

        for epoch in range(args.num_epochs):
            loader.reset_batch_pointer()
            state_ = sess.run(init_state)
            training_loss = 0

            for batch in range(num_batches):
                #state_ = sess.run(init_state)
                x, y = loader.next_batch_train()

                feed_dict = dict()
                feed_dict[inputs] = x
                feed_dict[targets] = y
                
                if ('RNN' in myCell.__name__):
                    feed_dict[init_state] = state_
                else:
                    feed_dict[init_state.c] = state_.c
                    feed_dict[init_state.h] = state_.h

                train_loss_, state_, _ = sess.run([loss, final_state, train_step], feed_dict=feed_dict)
                training_loss += train_loss_
            training_loss=training_loss/num_batches
            print('epoch:', epoch, 'loss:',  training_loss)
            training_losses.append(training_loss)
        saver.save(sess, os.path.join(args.save_dir, 'saved_model'))
    
    end_time = time.time()
    
    timer(start_time, end_time)
    return(training_losses)

# Run testing by getting the cross entropy loss on test batches.

In [12]:
def Tester(MyCell, num_batches=None):
    tf.reset_default_graph()
    with tf.variable_scope('placeholders'):
            inputs = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
            targets = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
    init_state, train_step, loss, final_state, saver, prob=network(MyCell,inputs,targets)

    # Define initialization
    
    loader= TextLoader(args.data_dir, batch_size=args.batch_size, seq_length=args.num_steps)

 
    if (num_batches is None):
        num_batches=loader.test_num_batches
    with tf.Session() as sess:

        # Load saved model
        saver.restore(sess, 'saved_model')
        state_ = sess.run(init_state)

        loader.reset_batch_pointer()
        
         # Get test error loss
        test_loss = 0
        print('num_batches',num_batches)

        for batch in range(num_batches):

            x, y = loader.next_batch_test()

            feed_dict = dict()
            feed_dict[inputs] = x
            feed_dict[targets] = y

            if ('RNN' in MyCell.__name__):
                feed_dict[init_state] = state_
            else:
                feed_dict[init_state.c] = state_.c
                feed_dict[init_state.h] = state_.h

            test_loss_, state_= sess.run([loss, final_state], feed_dict=feed_dict)
            test_loss += test_loss_
        test_loss=test_loss/num_batches
        print('test loss:',  test_loss)


    
    return(test_loss)

# Synthesize text using an initializing text running the network on the initializing text and then contiunuing with the final hidden state to simulate new text.

In [13]:
def Synthesize(MyCell,init_string="None"):
    tf.reset_default_graph()
    num_steps_bak=args.num_steps
    batch_size_bak=args.batch_size
    args.num_steps=1
    args.batch_size=1
    with tf.variable_scope('placeholders'):
            inputs = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
            targets = tf.placeholder(tf.int32, [args.batch_size, args.num_steps])
    init_state, train_step, loss, final_state, saver, prob=network(MyCell,inputs,targets)

    # Define initialization
    if (init_string is None):
        initialization = 'Where are you going today?'
    else:
        initialization = init_string
    loader= TextLoader(args.data_dir, batch_size=1, seq_length=1)

    forecast_data=np.array(list(map(loader.vocab_to_idx.get, initialization)))
    print(forecast_data)
    forecast_range = 100
    top_k=5
 
    with tf.Session() as sess:

        # Load saved model
        saver.restore(sess, 'saved_model')
        state_ = sess.run(init_state)

        # Run rnn on initialization data to get final hidden state before simulation
        state_ = sess.run(init_state)
        for i in range(forecast_data.shape[0]):

            feed_dict = dict()
            # Feed current predicted
            feed_dict[inputs] = forecast_data[i].reshape(args.batch_size, args.num_steps)
            if ('RNN' in MyCell.__name__):
                feed_dict[init_state] = state_
            else:
                feed_dict[init_state.c] = state_.c
                feed_dict[init_state.h] = state_.h
            # Get new hidden state and prediction probabilities
            predicted_prob, state_ = sess.run([prob, final_state], feed_dict=feed_dict)

        # last state of this step becomes first state of simulation

        for i in range(forecast_range):

            feed_dict = dict()
            # Feed current predicted
            feed_dict[inputs] = forecast_data[-args.num_steps:].reshape(args.batch_size, args.num_steps)
            if ('RNN' in MyCell.__name__):
                feed_dict[init_state] = state_
            else:
                feed_dict[init_state.c] = state_.c
                feed_dict[init_state.h] = state_.h
            predicted_prob, state_ = sess.run([prob, final_state], feed_dict=feed_dict)

            predicted_prob = predicted_prob.ravel()
            # Simulate from top top_k probs
            predicted_prob[np.argsort(predicted_prob)[:-top_k]] = 0
            predicted_prob = predicted_prob/np.sum(predicted_prob)
            sample = np.random.choice(args.num_chars, 1, p=predicted_prob)[0]


            forecast_data = np.hstack((forecast_data, sample))

    forecasted_chars = np.asarray([loader.idx_to_vocab[elem] for elem in forecast_data])

    print(''.join(forecasted_chars))
  
    args.num_steps=num_steps_bak
    args.batch_size=batch_size_bak

# Running the different models and testing them

In [10]:
re_lstm_basic = trainer(MyBasicLSTMCell)

train_num_batches 185
epoch: 0 loss: 2.375254158715944
epoch: 1 loss: 1.8126878319559871
epoch: 2 loss: 1.6422542526915267
epoch: 3 loss: 1.548723560410577
epoch: 4 loss: 1.4858037394446295
epoch: 5 loss: 1.4417319639309032
epoch: 6 loss: 1.408230753202696
epoch: 7 loss: 1.3803289381233421
epoch: 8 loss: 1.3581474445961617
epoch: 9 loss: 1.3408515459782369
epoch: 10 loss: 1.3245993962159028
epoch: 11 loss: 1.3103126751409995
epoch: 12 loss: 1.2975761716430252
epoch: 13 loss: 1.2865822212116138
epoch: 14 loss: 1.2747626349732684
epoch: 15 loss: 1.264664246584918
epoch: 16 loss: 1.2557935772715387
epoch: 17 loss: 1.2484358072280883
epoch: 18 loss: 1.2407760864979511
epoch: 19 loss: 1.2327337368114575
epoch: 20 loss: 1.225172453957635
epoch: 21 loss: 1.2180879947301504
epoch: 22 loss: 1.211738293879741
epoch: 23 loss: 1.2060794798103538
epoch: 24 loss: 1.2014588124043233
epoch: 25 loss: 1.1996361242758262
epoch: 26 loss: 1.1971267184695682
epoch: 27 loss: 1.1944619088559538
epoch: 28 loss

In [14]:
Tester(MyBasicLSTMCell)

there are preprocessed data - lets load it
[39  5  1  7  1  0  4  7  1  0 15  3 13  0 20  3  9  8 20  0  2  3 12  4
 15 44]
INFO:tensorflow:Restoring parameters from saved_model
num_batches 47
test loss: 1.4047210089703823


1.4047210089703823

In [15]:
Synthesize(MyBasicLSTMCell,"What is the meaning of life")

there are preprocessed data - lets load it
[39  5  4  2  0  9  6  0  2  5  1  0 14  1  4  8  9  8 20  0  3 18  0 11
  9 18  1]
INFO:tensorflow:Restoring parameters from saved_model
What is the meaning of lifen.

DUCHESS OF YORK:
Well, good father, this is a present days,
The play'd thy heatter, when you hav


In [19]:
re_lstm_lessgate=trainer(MyBasicLSTMCell_lessgate)

train_num_batches 185
epoch: 0 loss: 2.4314659956339244
epoch: 1 loss: 1.8905949940552582
epoch: 2 loss: 1.7162846378377965
epoch: 3 loss: 1.615376700581731
epoch: 4 loss: 1.5499827417167458
epoch: 5 loss: 1.501998496055603
epoch: 6 loss: 1.4657965608545251
epoch: 7 loss: 1.436786049121135
epoch: 8 loss: 1.4134604975983904
epoch: 9 loss: 1.3927866967948708
epoch: 10 loss: 1.3778861980180483
epoch: 11 loss: 1.3615304160762478
epoch: 12 loss: 1.3485511876441336
epoch: 13 loss: 1.336150902670783
epoch: 14 loss: 1.3274670323810063
epoch: 15 loss: 1.318680637591594
epoch: 16 loss: 1.3076662437335864
epoch: 17 loss: 1.299986473289696
epoch: 18 loss: 1.293395793760145
epoch: 19 loss: 1.2851229042620271
epoch: 20 loss: 1.2790844272922826
epoch: 21 loss: 1.275701347557274
epoch: 22 loss: 1.267823540842211
epoch: 23 loss: 1.2612911707646137
epoch: 24 loss: 1.2561672874399135
epoch: 25 loss: 1.2542885973646833
epoch: 26 loss: 1.2479632893124142
epoch: 27 loss: 1.2416488106186325
epoch: 28 loss: 1

In [16]:
Tester(MyBasicLSTMCell_lessgate)

there are preprocessed data - lets load it
INFO:tensorflow:Restoring parameters from saved_model
num_batches 47
test loss: 1.3992189143566376


1.3992189143566376

In [15]:
re_rnn_basic = trainer(MyBasicRNNCell)

train_num_batches 185
epoch: 0 loss: 2.2866439722679757
epoch: 1 loss: 1.8399305562715274
epoch: 2 loss: 1.6943824864722588
epoch: 3 loss: 1.6129409409858084
epoch: 4 loss: 1.5597342678018518
epoch: 5 loss: 1.5227948936256201
epoch: 6 loss: 1.495543941291603
epoch: 7 loss: 1.4738708856943492
epoch: 8 loss: 1.4562455061319712
epoch: 9 loss: 1.4417609330770131
epoch: 10 loss: 1.4296678446434639
epoch: 11 loss: 1.4196067913158519
epoch: 12 loss: 1.4109132090130367
epoch: 13 loss: 1.4029981858021505
epoch: 14 loss: 1.3962290828292434
epoch: 15 loss: 1.390509927594984
epoch: 16 loss: 1.3851184748314522
epoch: 17 loss: 1.3802465155317976
epoch: 18 loss: 1.375983655130541
epoch: 19 loss: 1.3717353930344454
epoch: 20 loss: 1.3677776613750974
epoch: 21 loss: 1.3644523942792737
epoch: 22 loss: 1.3617173523516268
epoch: 23 loss: 1.3598538508286346
epoch: 24 loss: 1.3585559806308232
epoch: 25 loss: 1.3561777404836706
epoch: 26 loss: 1.3540037986394522
epoch: 27 loss: 1.3522032595969535
epoch: 28 l

In [16]:
Tester(MyBasicRNNCell)

there are preprocessed data - lets load it
INFO:tensorflow:Restoring parameters from saved_model
num_batches 47
test loss: 1.4636335271470091


1.4636335271470091

In [17]:
Synthesize(MyBasicRNNCell,"What is the meaning of life")

there are preprocessed data - lets load it
[39  5  4  2  0  9  6  0  2  5  1  0 14  1  4  8  9  8 20  0  3 18  0 11
  9 18  1]
INFO:tensorflow:Restoring parameters from saved_model
What is the meaning of lifer
And told to the bride the king of command thee
As thou sell mine earth.
Where in her son,
And man 


In [17]:
re_rnn_basic_d2 = trainer(MyBasicRNNCell_d2)

train_num_batches 185
epoch: 0 loss: 2.271846804103336
epoch: 1 loss: 1.8089590768556336
epoch: 2 loss: 1.6572276076755008
epoch: 3 loss: 1.5718503616951607
epoch: 4 loss: 1.5170617232451569
epoch: 5 loss: 1.4790298552126497
epoch: 6 loss: 1.450774130950103
epoch: 7 loss: 1.4293345013180294
epoch: 8 loss: 1.412054891199679
epoch: 9 loss: 1.3972392713701403
epoch: 10 loss: 1.3849842425939198
epoch: 11 loss: 1.3753085535925789
epoch: 12 loss: 1.3667957331683185
epoch: 13 loss: 1.3601067504367312
epoch: 14 loss: 1.3553066330987054
epoch: 15 loss: 1.3503588805327544
epoch: 16 loss: 1.3472831526318112
epoch: 17 loss: 1.3440541776450905
epoch: 18 loss: 1.3393264255008182
epoch: 19 loss: 1.3336764258307379
epoch: 20 loss: 1.3311266744458998
epoch: 21 loss: 1.3299312829971313
epoch: 22 loss: 1.3276082122648085
epoch: 23 loss: 1.323850263776006
epoch: 24 loss: 1.3204784064679533
epoch: 25 loss: 1.3190409705445574
epoch: 26 loss: 1.3163784536155494
epoch: 27 loss: 1.3159131681596912
epoch: 28 lo

In [18]:
Tester(MyBasicRNNCell_d2)

there are preprocessed data - lets load it
[39  5  1  7  1  0  4  7  1  0 15  3 13  0 20  3  9  8 20  0  2  3 12  4
 15 44]
INFO:tensorflow:Restoring parameters from saved_model
num_batches 47
test loss: 1.4272671354577897


1.4272671354577897

In [30]:
re_rnn_basic_d4 = trainer(MyBasicRNNCell_d4)

train_num_batches 185
epoch: 0 loss: 2.408355314022786
epoch: 1 loss: 1.8452126780071774
epoch: 2 loss: 1.6803465772319484
epoch: 3 loss: 1.5881870450200262
epoch: 4 loss: 1.528356367188531
epoch: 5 loss: 1.486127984201586
epoch: 6 loss: 1.4557582365499961
epoch: 7 loss: 1.4338586981232102
epoch: 8 loss: 1.416147020056441
epoch: 9 loss: 1.404949699865805
epoch: 10 loss: 1.3931776439821397
epoch: 11 loss: 1.380185132413297
epoch: 12 loss: 1.3715257258028597
epoch: 13 loss: 1.3649000328940315
epoch: 14 loss: 1.3596686053920437
epoch: 15 loss: 1.3541683390333845
epoch: 16 loss: 1.3500864331786697
epoch: 17 loss: 1.3480135717907467
epoch: 18 loss: 1.34493774401175
epoch: 19 loss: 1.341222183124439
epoch: 20 loss: 1.3374746870350194
epoch: 21 loss: 1.3403148825104172
epoch: 22 loss: 1.3379634831402754
epoch: 23 loss: 1.3488434411383965
epoch: 24 loss: 1.3350426081064586
epoch: 25 loss: 1.3343019118180146
epoch: 26 loss: 1.3386835620209978
epoch: 27 loss: 1.3370027116827063
epoch: 28 loss: 1

In [31]:
Tester(MyBasicRNNCell_d4)

there are preprocessed data - lets load it
INFO:tensorflow:Restoring parameters from saved_model
num_batches 47
test loss: 1.4106874440578705


1.4106874440578705